In [1]:
import tensorflow_datasets as tfds
from tensorflow.keras.layers import *
from tensorflow.keras import Model
import tensorflow as tf

from sklearn.model_selection import train_test_split
import numpy as np

## Load data

In [2]:
dataset, info = tfds.load(
    "uc_merced",
    split=['train'],
    with_info=True,
    batch_size=-1

)

In [3]:
info

tfds.core.DatasetInfo(
    name='uc_merced',
    full_name='uc_merced/2.0.0',
    description="""
    UC Merced is a 21 class land use remote sensing image dataset, with 100 images
    per class. The images were manually extracted from large images from the USGS
    National Map Urban Area Imagery collection for various urban areas around the
    country. The pixel resolution of this public domain imagery is 0.3 m.
    
    While most images are 256x256 pixels, there are 44 images with different shape.
    """,
    homepage='http://weegee.vision.ucmerced.edu/datasets/landuse.html',
    data_path='dataset/uc_merced/2.0.0',
    file_format=tfrecord,
    download_size=317.07 MiB,
    dataset_size=238.63 MiB,
    features=FeaturesDict({
        'filename': Text(shape=(), dtype=string),
        'image': Image(shape=(None, None, 3), dtype=uint8),
        'label': ClassLabel(shape=(), dtype=int64, num_classes=21),
    }),
    supervised_keys=('image', 'label'),
    disable_shuffling=False,
  

## Data processing

In [4]:
def preprocess(image):
    image = tf.cast(image, tf.float32)
    image = (image/127.5) - 1
    image = tf.image.resize(image, (64, 64))
    return image

In [5]:
images = []
for i in tfds.as_numpy(dataset[0])[0]:
    images.append(preprocess(i))

images = np.array(images)
labels = tfds.as_numpy(dataset[0])[1].astype('uint32')

In [6]:
images.shape, labels.shape

((2100, 64, 64, 3), (2100,))

In [7]:
X_train, X_test, y_train, y_test = train_test_split(images, labels, test_size=0.1, random_state=42)
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((1890, 64, 64, 3), (210, 64, 64, 3), (1890,), (210,))

## Build model

In [8]:
def inception_layer(x, filters):

    f1, f2_1, f2_2, f3_1, f3_2, f4 = filters

    x1 = Conv2D(f1, (1, 1), padding='same', activation='relu')(x)

    x2 = Conv2D(f2_1, (1, 1), padding='same', activation='relu')(x)
    x2 = Conv2D(f2_2, (3, 3), padding='same', activation='relu')(x2)

    x3 = Conv2D(f3_1, (1, 1), padding='same', activation='relu')(x)
    x3 = Conv2D(f3_2, (5, 5), padding='same', activation='relu')(x3)

    x4 = MaxPool2D((3, 3), strides=(1, 1), padding='same')(x)
    x4 = Conv2D(f4, (1, 1), padding='same', activation='relu')(x4)

    output = concatenate([x1, x2, x3, x4], axis=3)

    return output

In [9]:
def InceptionNet(input_shape=(224,224,3), num_classes=10):
    x_input = Input(input_shape)

    x = UpSampling2D(size=(7,7))(x_input)

    x = Conv2D(64, (7, 7), padding='same', strides=(2, 2), activation='relu')(x)
    x = MaxPool2D((3, 3), padding='same', strides=(2, 2))(x)
    x = Conv2D(64, (1, 1), padding='same', strides=(1, 1), activation='relu')(x)
    x = Conv2D(192, (3, 3), padding='same', strides=(1, 1), activation='relu')(x)
    x = MaxPool2D((3, 3), padding='same', strides=(2, 2))(x)

    x = inception_layer(x, [64, 96, 128, 16, 32, 32])
    x = inception_layer(x, [128, 128, 192, 32, 96, 64])

    x = MaxPool2D((3, 3), padding='same', strides=(2, 2))(x)

    x = inception_layer(x, [192, 96, 208, 16, 48, 64])

    x1 = AveragePooling2D(5, strides=3)(x)
    x1 = Conv2D(128, (1, 1), padding='same', activation='relu')(x1)
    x1 = Flatten()(x1)
    x1 = Dense(1024, activation='relu')(x1)
    x1 = Dense(num_classes, activation='softmax')(x1)

    x = inception_layer(x, [160, 112, 224, 24, 64, 64])
    x = inception_layer(x, [128, 128, 256, 24, 64, 64])
    x = inception_layer(x, [112, 144, 288, 32, 64, 64])

    x2 = AveragePooling2D((5, 5), strides=3)(x)
    x2 = Conv2D(128, (1, 1), padding='same', activation='relu')(x2)
    x2 = Flatten()(x2)
    x2 = Dense(1024, activation='relu')(x2)
    x2 = Dense(num_classes, activation='softmax')(x2)

    x = inception_layer(x, [256, 160, 320, 32, 128, 128])

    x = MaxPool2D((3, 3), padding='same', strides=(2, 2))(x)

    x = inception_layer(x, [256, 160, 320, 32, 128, 128])
    x = inception_layer(x, [384, 192, 384, 48, 128, 128])
    x = AveragePooling2D()(x)

    x = Flatten()(x)
    x = Dense(num_classes, activation='softmax')(x)

    model = Model(inputs=x_input, outputs=[x, x2, x1], name='InceptionNet')
    return model

In [10]:
inception_model = InceptionNet(input_shape=X_train.shape[1:], num_classes=21)
inception_model.summary()

Model: "InceptionNet"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 64, 64, 3)]  0           []                               
                                                                                                  
 up_sampling2d (UpSampling2D)   (None, 448, 448, 3)  0           ['input_1[0][0]']                
                                                                                                  
 conv2d (Conv2D)                (None, 224, 224, 64  9472        ['up_sampling2d[0][0]']          
                                )                                                                 
                                                                                                  
 max_pooling2d (MaxPooling2D)   (None, 112, 112, 64  0           ['conv2d[0][0]']      

In [11]:
inception_model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

In [12]:
inception_model.fit(
    X_train, y_train,
    validation_data=(X_test, y_test),
    epochs=30
)

Epoch 1/30
60/60 [==============================] - 63s 457ms/step - loss: 8.0701 - dense_4_loss: 2.7431 - dense_3_loss: 2.7194 - dense_1_loss: 2.6077 - dense_4_accuracy: 0.1026 - dense_3_accuracy: 0.1116 - dense_1_accuracy: 0.1704 - val_loss: 6.6143 - val_dense_4_loss: 2.2884 - val_dense_3_loss: 2.3008 - val_dense_1_loss: 2.0252 - val_dense_4_accuracy: 0.2095 - val_dense_3_accuracy: 0.1952 - val_dense_1_accuracy: 0.2619
Epoch 2/30
60/60 [==============================] - 22s 364ms/step - loss: 6.3163 - dense_4_loss: 2.2476 - dense_3_loss: 2.1698 - dense_1_loss: 1.8989 - dense_4_accuracy: 0.2228 - dense_3_accuracy: 0.2714 - dense_1_accuracy: 0.3704 - val_loss: 5.9124 - val_dense_4_loss: 2.3453 - val_dense_3_loss: 1.9182 - val_dense_1_loss: 1.6488 - val_dense_4_accuracy: 0.2429 - val_dense_3_accuracy: 0.3619 - val_dense_1_accuracy: 0.4524
Epoch 3/30
60/60 [==============================] - 22s 366ms/step - loss: 5.3024 - dense_4_loss: 1.9366 - dense_3_loss: 1.8212 - dense_1_loss: 1.5446